# Referenzbeispiel 4

# Zeichenketten

Der Zusatzstoff der vierten Übungseinheit gegenüber den ersten drei Einheiten umfasst die Kapitel 15 bis 16 im Buch “Programmieren in C”.

## 1 Zeichen ersetzen

Es soll ein Programm entwickelt werden, das Zeichen in einem Text durch andere ersetzen kann. Im folgenden Beispiel werden alle Kleinbuchstaben gegen Großbuchstaben ausgetauscht. Bereits vorhandene Großbuchstaben bleiben erhalten. Hierbei entspricht ein Zeichen in `Ersetzen` immer einem Zeichen in `Suchen`:
```
Suchen:   abcdefghijklmnopqrstuvwxzy
Ersetzen: ABCDEFGHIJKLMNOPQRSTUVWXYZ
Dies ist ein Beispieltext.
DIES IST EIN BEISPIELTEXT.
```
Ein anderes Beispiel: (e durch 3, i durch 1 ersetzen)
```
Suchen:   ei
Ersetzen: 31
Dies ist ein Beispieltext.
D13s 1st 31n B31sp13lt3xt.
```

### 1.1 Einlesefunktion für ganze Zeilen

Wir wollen nun ganze Zeilen einlesen, damit auch Leerzeichen ersetzt werden können. `scanf` liest aber nur bis zum ersten Leerzeichen oder Zeilenumbruch. Daher müssen wir eine eigene Funktion schreiben. (Es gibt eine Funktion `getline` in C. Diese benötigt allerdings dynamischen Speicher, welcher erst in Programmieren 2 behandelt wird.)

Die folgende Funktion `getLine`(**Achtung**: großes L) kann nützlich sein, wenn dynamischer Speicher vermieden werden soll:

In [ ]:
/* Einelesefunktion, die erst beim Zeichen '\n' aufhört zu lesen */
#include <stdio.h>

/* num Zeichen einlesen und in str speichern.
    Gibt die Anzahl der gelesenen Zeichen zurück */
long getLine(char str[], long num) {
    char c = 0;
    long counter = 0;
    
    /* wenn num oder str nicht valide sind */
    if(num <= 0 || str == 0) {
        return 0;
    }
    
    /* So lange kein \n kommt, Zeichen einlesen */
    while((c = getchar()) != '\n') {
        str[counter] = c;
        ++counter;
        
        /* Wenn die Zeichenkette voll ist, aber kein '\n' gefunden wurde */
        if(counter == num) {
            printf("Die eingegeben Zeichenkette ist zu lange! Nochmals eingeben\n");
            counter = 0; /* Zähler zurücksetzen */
            while(getchar() != '\n'); /* Input komplett leeren */
        }
    }
    
    /* Nullcharakter setzen */
    str[counter] = '\0';
    
    return counter;
}

int main() {
    char str[20];
    
    getLine(str, 20);
    
    printf("%s\n", str);
    
    return 0;
}

### 1.2 Beispielprogramm

Das folgende Programm ersetzt, wie beschrieben, Zeichen die der Benutzer eingibt mit anderen Zeichen des Benutzers. Die Längen der `Suchen` und `Ersetzen` Felder müssen hierbei natürlich gleich sein!

In [ ]:
#include <stdio.h>
#include <string.h>

#define SEARCHLEN 256
#define TEXTLEN 100

/* Diese Deklarationen würden üblicherweise in
    einer eigenen Header-Datei (.h) stehen */
long getLine(char[], long);
char doReplace(char, char[], char[]);

/* In main.c sollte nur int main() definiert werden */
int main() {
    char search[SEARCHLEN];
    char replace[SEARCHLEN];
    char text[TEXTLEN];
    
    /* Durch das Einlesen mit getLine können auch Leerzeichen ersetzt werden! */
    printf("Geben Sie die Zeichen ein, die ersetzt werden sollen:\n");
    getLine(search, SEARCHLEN);
    
    /* So lange nicht gleich viele replace Zeichen, wie search Zeichen definiert sind */
    do {
        printf("Geben Sie die Zeichen ein, die eingefügt werden sollen:\n");
        getLine(replace, SEARCHLEN);
    } while(strlen(replace) != strlen(search));
    
    printf("Geben Sie den Text ein, der durchsucht werden soll:\n");
    getLine(text, TEXTLEN);
    
    {
        char *pointer = text;
        while(*pointer != '\0') {
            *pointer = doReplace(*pointer, search, replace);
            ++pointer;
        }
    }
    
    /* Ersetzten Text ausgeben */
    printf("%s\n", text);
    
    return 0;
}

/* Diese Definitionen würden üblicherweise
    in einer eigenen .c Datei stehen */
char doReplace(char character, char search[], char replace[]) {
    /* Zeichen in search finden */
    char *pos = strchr(search, character);
    
    /* Wenn nicht gefunden */
    if(pos == NULL) {
        return character;
    } else {
        return replace[pos - search]; /* Das entsprechende Zeichen in replace zurück geben */
    }
}

long getLine(char str[], long num) {
    char c = 0;
    long counter = 0;
    
    if(num <= 0 || str == 0) {
        return 0;
    }
    
    while((c = getchar()) != '\n') {
        str[counter] = c;
        ++counter;
        
        if(counter == num) {
            printf("Die eingegeben Zeichenkette ist zu lange! Nochmals eingeben\n");
            counter = 0;
            while(getchar() != '\n');
        }
    }   
    str[counter] = '\0';
    
    return counter;
}

## 2 Fragen zum Selbststudium

* Wie viele Zeichen benötigt die Zeichenkette "Test" im Speicher?
* Warum muss der Funktion `getLine` die Länge übergeben werden?
* Wenn die obigen Funktionen Felder aus Zahlen und nicht aus Zeichen verarbeiten würden, müsste man dann trotzdem die Feldlängen übergeben?

## 3 Effiziente Implementierung Beispielprogramm

Im oben gezeigten Programm muss jedes Zeichen im Feld `search` gesucht werden um es zu ersetzen. Wenn viel Text ersetzt werden soll, kann das lange dauern und beinhaltet unnötige Arbeit, da zum Beispiel jedes `a` erneut gesucht wird, obwohl es vielleicht vorher schon gefunden wurde.

Hierzu gibt es zwei mögliche Verbesserungen:

1. `text` wird mehrmals durchgangen und immer nur ein Zeichen mit einem anderen ersetzt. <br>
Dadurch kann ein langer Text effizienter ersetzt werden. Trotzdem muss `text` jetzt mehrmals angesehen werden.

2. Die effizienteste Methode ist eine Übersetzungstabelle:<br>
Hierbei wird jedem einzelnen Zeichen ein anderes zugewiesen. Wenn es nicht ersetzt werden soll, ist es einfach das gleiche Zeichen. Wenn zum Beispiel `'a'` mit `'b'` ersetzt werden soll, steht am Index `'a'` der Tabelle, das Zeichen `'b'`. Daher ist dieser Index der Tabelle wie folgt definiert:    
`Tabelle['a'] = 'b';`    
`'a'` hat in der ASCII Tabelle den Wert `97`, daher entspricht das also dem 98sten Element der Tabelle. Nun muss nur jedem Element der Tabelle das entsprechende Zeichen zugewiesen werden und alle Zeichen bei einem einzigen Durchlauf von `text` ersetzt werden.

## 4 Weitere Übungsbeispiele

### Zeichen ignorieren

Erstellen Sie ein Programm, das bestimme ASCII-Zeichen ignoriert:
```
Überlesene Zeichen: test,
Texteingabe: Erstellen Sie ein Programm, das Zeichen aus einem eingegebenen Text
ignoriert.
Textausgabe: Erlln Si in Programm da Zichn au inm inggbnn Tx ignorir.
```
Erweitern Sie das Programm so, dass auch die dazugehörigen Großbuchstaben ignoriert werden.

### Zahlen sortieren

Lesen Sie 10 ganze Zahlen ein und geben Sie diese sortiert wieder aus. Bauen Sie eine Abfrage ein, ob die Ausgabe auf- oder absteigend sortiert werden soll.

### Häufigkeit von Text

Erstellen Sie ein Programm, welches die Häufigkeit einer eingelesenen Zeichenkette in einem Text bestimmt:
```
Zeichenkette: ei
Texteingabe: Lesen Sie 10 ganze Zahlen ein und geben Sie diese sortiert wieder
aus. Bauen Sie eine Abfrage ein, ob die Ausgabe auf- oder
absteigend sortiert werden soll.
Textausgabe: Die Zeichenkette "ei" wurde 4 mal gefunden.
```

### Ein einfacher Linter

Ein Linter ist ein Programm, dass Quelltext analysiert und automatisch Fehler erkennt und diese beheben kann. Sie sollen einen simplen Linter für die Sprache C programmieren:
* Nach `;`, `{`, `}` soll automatisch ein `'\n'` eingefügt werden
* Vor und nach `=`, `+`, `-` `*`, `/` sollen Leerzeichen eingefügt werden
* Nach `,` soll ein Leerzeichen eingefügt werden

Beispiel:
```
Texteingabe: int main(){long a=5,b=6;printf("%ld",a+b);return 0;}
Textausgabe:
int main(){
long a = 5, b = 6;
printf("%ld", a + b);
return 0;
}
```

Nun stimmt die Einrückung in diesem Beispiel noch nicht. Erweitern Sie ihr Programm:
* Für jedes `{` sollen alle folgenden Zeilen um 4 Leerzeichen mehr eingerückt werden. Für jedes `}` wieder 4 weniger. Nach zwei `{` beginnen also alle Zeilen mit 8 Leerzeichen.
* Nach jedem `if` und `else` soll ein Zeilenumbruch folgen, wenn kein `{` folgt
* Ein Fehler soll ausgegeben werden, wenn nicht alle Klammern geschlossen sind

**TIPP**: Behandeln Sie jede dieser Ersetzungen einzeln mit eigenen Funktionen. Dadruch können Sie ein großes Problem in viele verständliche kleine Probleme zerlegen.